In [1]:
from tensorflow import keras
keras.__version__

'2.4.0'

To start using `zoo.orca`, we need to first initialize orca context. Here we specify local or distributed mode. In this example, we choose the local mode.

In [2]:
from zoo.orca import init_orca_context, stop_orca_context
from zoo.orca import OrcaContext

OrcaContext.log_output = True # recommended to set it to True when running Analytics Zoo in Jupyter notebook (this will display terminal's  stdout and stderr in the Jupyter notebook).

cluster_mode = "local"

if cluster_mode == "local":  
    init_orca_context(cluster_mode="local", cores=4) # run in local mode
elif cluster_mode == "k8s":  
    init_orca_context(cluster_mode="k8s", num_nodes=2, cores=2) # run on K8s cluster
elif cluster_mode == "yarn":  
    init_orca_context(cluster_mode="yarn-client", num_nodes=2, cores=2) # run on Hadoop YARN cluster

Initializing orca context
Current pyspark location is : /intern/spark/spark-2.4.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /home/zhenhao/analytics-zoo/zoo/target/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.10.0-SNAPSHOT-dist-all/lib/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.10.0-SNAPSHOT-jar-with-dependencies.jar pyspark-shell 


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/zhenhao/analytics-zoo/zoo/target/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.10.0-SNAPSHOT-dist-all/lib/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.10.0-SNAPSHOT-jar-with-dependencies.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/intern/spark/spark-2.4.3-bin-hadoop2.7/jars/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2021-01-26 12:08:29 WARN  Utils:66 - Your hostname, intern01 resolves to a loopback address: 127.0.1.1; using 10.239.44.107 instead (on interface eno1)
2021-01-26 12:08:29 WARN  Utils:66 - Set SPARK_LOCAL_IP if you need to bind to another address
2021-01-26 12:08:29 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
cls.getname: com.intel.analytics.bigdl.python.api.Sample
BigDLBasePickler registering: bigdl.util.common  Sample
cls.getname: com.intel.analytics.bigdl.python.api.EvaluatedResult
BigDLBasePickler registering: bigdl.util.common  EvaluatedResult
cls.getname: com.intel.analytics.bigdl.python.api.JTensor
BigDLBasePickler registering: bigdl.util.common  JTensor
cls.getname: com.intel.analytics.bigdl.python.api.JActivity
Successfully got a SparkContextBigDLBasePickler registering: bigdl.util.common  JActivity




User settings:

   KMP_AFFINITY=granularity=fine,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=1

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_INIT_WAIT=2048
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NEXT_WAIT=1024
   KMP_NUM_L

# Predicting house prices: a regression example
This notebook contains the code samples found in Chapter 3, Section 6 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

In our two previous examples, we were considering classification problems, where the goal was to predict a single discrete label of an input data point. Another common type of machine learning problem is "regression", which consists of predicting a continuous value instead of a discrete label. For instance, predicting the temperature tomorrow, given meteorological data, or predicting the time that a software project will take to complete, given its specifications.

Do not mix up "regression" with the algorithm "logistic regression": confusingly, "logistic regression" is not a regression algorithm, it is a classification algorithm.

## The Boston Housing Price dataset

We will be attempting to predict the median price of homes in a given Boston suburb in the mid-1970s, given a few data points about the suburb at the time, such as the crime rate, the local property tax rate, etc.

The dataset we will be using has another interesting difference from our two previous examples: it has very few data points, only 506 in total, split between 404 training samples and 102 test samples, and each "feature" in the input data (e.g. the crime rate is a feature) has a different scale. For instance some values are proportions, which take a values between 0 and 1, others take values between 1 and 12, others between 0 and 100...

Let's take a look at the data:

In [3]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()
train_data.shape

Using TensorFlow backend.


(404, 13)

In [4]:
test_data.shape

(102, 13)

We can also extract the information of our dataset using `numpy`.

In [5]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)


As you can see, we have 404 training samples and 102 test samples. The data comprises 13 features. The 13 features in the input data are as follow:

1. Per capita crime rate.
1. Proportion of residential land zoned for lots over 25,000 square feet.
1. Proportion of non-retail business acres per town.
1. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
1. Nitric oxides concentration (parts per 10 million).
1. Average number of rooms per dwelling.
1. Proportion of owner-occupied units built prior to 1940.
1. Weighted distances to five Boston employment centres.
1. Index of accessibility to radial highways.
1. Full-value property-tax rate per $10,000.
1. Pupil-teacher ratio by town.
1. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
1. % lower status of the population.

The targets are the median values of owner-occupied homes, in thousands of dollars:

In [6]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

The prices are typically between \$ 10,000 and \$ 50,000. If that sounds cheap, remember this was the mid-1970s, and these prices are not inflation-adjusted.

## Preparing the data

It would be problematic to feed into a neural network values that all take wildly different ranges. The network might be able to automatically adapt to such heterogeneous data, but it would definitely make learning more difficult. A widespread best practice to deal with such data is to do feature-wise normalization: for each feature in the input data (a column in the input data matrix), we will subtract the mean of the feature and divide by the standard deviation, so that the feature will be centered around 0 and will have a unit standard deviation. We can directly use the numbers calculated above.

### Fit the data into Orca Estimator
To fit the data with Orca estimator, we will need to create two data creator functions that transform the training data and test data into `tf.data.Dataset`. The function takes in one argument `config`, which is a dictionary where we can put in necessary information, such as mean and standard deviation of training set and the batch size.

### Validating our approach using K-fold validation
To evaluate our network while we keep adjusting its parameters (such as the number of epochs used for training), we could simply split the data into a training set and a validation set, as we were doing in our previous examples. However, because we have so few data points, the validation set would end up being very small (e.g. about 100 examples). A consequence is that our validation scores may change a lot depending on which data points we choose to use for validation and which we choose for training, i.e. the validation scores may have a high variance with regard to the validation split. This would prevent us from reliably evaluating our model.

The best practice in such situations is to use K-fold cross-validation. It consists of splitting the available data into K partitions (typically K=4 or 5), then instantiating K identical models, and training each one on K-1 partitions while evaluating on the remaining partition. The validation score for the model used would then be the average of the K validation scores obtained.

In terms of code, this is straightforward.

In [7]:
import tensorflow as tf
import numpy as np

def preprocess(data):
    """
    Map the original data type to tf.data.Dataset of shape ((13, 1), ()),
    where the first tensor is the features and the second tensor is the 
    fit target.
    """
    
    data['features'] = tf.cast(tf.reshape(data['features'], (13, 1)), dtype=tf.float32)
    data['label'] = tf.cast(data['label'], dtype=tf.float32)
    return data['features'], data['label']


def train_data_creator(config):
    # Load data from dataset
    (train_data, train_targets), _ =  boston_housing.load_data()
        
    # Data Standardizing
    train_data -= config["mean"]
    train_data /= config["std"]
    
    # Perform K-fold Validation if config["K-fold"] is true
    if config["K-fold"]:
        k = config["k"] # total number of folds
        i = config["i"] # current fold
        num_val_samples = len(train_data) // k
        partial_train_data = np.concatenate(
            [train_data[:i * num_val_samples],
             train_data[(i + 1) * num_val_samples:]],
            axis=0)
        partial_train_targets = np.concatenate(
            [train_targets[:i * num_val_samples],
             train_targets[(i + 1) * num_val_samples:]],
            axis=0)
    
    # Transform data into tf.data.Dataset and preprocess it
    if config["K-fold"]:
        tensors = {'features': partial_train_data, 'label': partial_train_targets}
    else:
        tensors = {'features': train_data, 'label': train_targets}
    train_dataset = tf.data.Dataset.from_tensor_slices(tensors)
    train_dataset = train_dataset.repeat()
    train_dataset = train_dataset.map(preprocess)
    train_dataset = train_dataset.shuffle(1000)
    train_dataset = train_dataset.batch(config["batch_size"])
    return train_dataset
    

def val_data_creator(config):
    # Load data from dataset
    if config["K-fold"]:
        (train_data, train_targets), _ =  boston_housing.load_data()
        
        # Data Standardizing
        train_data -= config["mean"]
        train_data /= config["std"]
    else: 
        _, (test_data, test_targets) = boston_housing.load_data()
        
        # Data Standardizing
        test_data -= config["mean"]
        test_data /= config["std"]
    
    
    # Perform K-fold Validation
    if config["K-fold"]:
        k = config["k"] # total number of folds
        i = config["i"] # current fold
        num_val_samples = len(train_data) // k
        val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    # Transform data into tf.data.Dataset and preprocess it
    if config["K-fold"]:
        tensors = {'features': val_data, 'label': val_targets}
    else:
        tensors = {'features': test_data, 'label': test_targets}
    test_dataset = tf.data.Dataset.from_tensor_slices(tensors)
    test_dataset = test_dataset.repeat()
    test_dataset = test_dataset.map(preprocess)
    test_dataset = test_dataset.batch(config["batch_size"])
    return test_dataset


Note that the quantities that we use for normalizing the test data have been computed using the training data. We should never use in our workflow any quantity computed on the test data, even for something as simple as data normalization.

## Building our network
Because so few samples are available, we will be using a very small network with two hidden layers, each with 64 units. In general, the less training data you have, the worse overfitting will be, and using a small network is one way to mitigate overfitting.

In [8]:
from tensorflow.keras import layers

def build_model(config):
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    model = keras.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(13, 1)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer=keras.optimizers.RMSprop(), loss='mse', metrics=['mae'])
    return model

Our network ends with a single unit, and no activation (i.e. it will be linear layer). This is a typical setup for scalar regression (i.e. regression where we are trying to predict a single continuous value). Applying an activation function would constrain the range that the output can take; for instance if we applied a sigmoid activation function to our last layer, the network could only learn to predict values between 0 and 1. Here, because the last layer is purely linear, the network is free to learn to predict values in any range.

Note that we are compiling the network with the `mse` loss function -- Mean Squared Error, the square of the difference between the predictions and the targets, a widely used loss function for regression problems.

We are also monitoring a new metric during training: `mae`. This stands for Mean Absolute Error. It is simply the absolute value of the difference between the predictions and the targets. For instance, a MAE of 0.5 on this problem would mean that our predictions are off by \$500 on average.

## Fit with Orca Estimator
We can create the estimator with the following

And we can fit our data using the estimator. Let's try training the network for a bit longer: 500 epochs. To keep a record of how well the model did at each epoch, we will modify our training loop to save the per-epoch validation score log:

In [9]:
from tensorflow.keras import backend as K

# Some memory clean-up
K.clear_session()

In [10]:
from zoo.orca.learn.tf2 import Estimator

k = 4
batch_size = 1
num_epochs = 100
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    mae_history = [] # storing all validation MAE
    est = Estimator.from_keras(model_creator=build_model,
                               config={},
                               workers_per_node=1,
                               verbose=0)
    stats = est.fit(train_data_creator, 
                        epochs=num_epochs,
                        batch_size=batch_size,
                        steps_per_epoch=450 // batch_size,
                        validation_data=val_data_creator,
                        validation_steps=150 // batch_size,
                        data_config={"mean": mean, "std": std, "batch_size": batch_size, "k": k, "i": i, "K-fold": True},
                        verbose=0)
    
    mae_history = stats['train_val_mae']

    est.shutdown()
    all_mae_histories.append(mae_history)

2021-01-26 12:08:32,766	INFO resource_spec.py:212 -- Starting Ray with 17.63 GiB memory available for workers and up to 8.82 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


processing fold # 0
{'node_ip_address': '10.239.44.107', 'redis_address': '10.239.44.107:27702', 'object_store_address': '/tmp/ray/session_2021-01-26_12-08-32_765084_32636/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2021-01-26_12-08-32_765084_32636/sockets/raylet', 'webui_url': None, 'session_dir': '/tmp/ray/session_2021-01-26_12-08-32_765084_32636'}
(pid=300) 2021-01-26 12:08:33.741656: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
(pid=300) 2021-01-26 12:08:33.741685: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(pid=300) 2021-01-26 12:08:34.589902: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or 

processing fold # 2
(pid=32766) 2021-01-26 12:10:06.878131: W tensorflow/core/common_runtime/eager/context.cc:566] Unable to destroy server_ object, so releasing instead. Servers don't support clean shutdown.
(pid=301) 2021-01-26 12:10:07.036495: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
(pid=301) 2021-01-26 12:10:07.036525: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(pid=301) 2021-01-26 12:10:07.872133: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
(pid=301) 2021-01-26 12:10:07.872154: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
(pid=301) 2021-01-

(pid=32767) WARNING:tensorflow:From /home/zhenhao/.local/lib/python3.7/site-packages/tensorflow/python/data/ops/multi_device_iterator_ops.py:601: get_next_as_optional (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
(pid=32767) Instructions for updating:
(pid=32767) Use `tf.data.Iterator.get_next_as_optional()` instead.


In [11]:
#est.shutdown()
print(stats)

{'train_loss': 67.70459747314453, 'train_mae': 5.982419490814209, 'train_val_loss': 117.83595275878906, 'train_val_mae': 7.448419570922852}


And we can see that the loss significantly decreases.

In [12]:
print(np.average(all_mae_histories))

6.626116514205933


Once we are done tuning other parameters of our model (besides the number of epochs, we could also adjust the size of the hidden layers), we can train a final "production" model on all of the training data, with the best parameters, then look at its performance on the test data:

In [13]:
batch_size = 16
num_epochs = 80
# Get a fresh, compiled model.
est = Estimator.from_keras(model_creator=build_model,
                           config={},
                           workers_per_node=1,
                           verbose=0)
# Train it on the entirety of the data.
stats = est.fit(train_data_creator, 
                epochs=num_epochs,
                batch_size=batch_size,
                steps_per_epoch=600 // batch_size,
                validation_data=val_data_creator,
                validation_steps=100 // batch_size,
                data_config={"mean": mean, "std": std, "batch_size": batch_size, "K-fold": False},
                verbose=0)
stats = est.evaluate(val_data_creator, 
                     data_config={"mean": mean, "std": std, "batch_size": batch_size, "K-fold": False},
                     num_steps=100 // batch_size)
est.shutdown()

(pid=32767) 2021-01-26 12:11:39.298260: W tensorflow/core/common_runtime/eager/context.cc:566] Unable to destroy server_ object, so releasing instead. Servers don't support clean shutdown.
(pid=3339) 2021-01-26 12:11:39.833144: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
(pid=3339) 2021-01-26 12:11:39.833171: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(pid=3339) 2021-01-26 12:11:40.675561: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
(pid=3339) 2021-01-26 12:11:40.675582: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
(pid=3339) 2021-01-26 12:11:40.675

In [14]:
print(stats)

{'validation_loss': 75.66140747070312, 'validation_mae': 6.230186462402344}


We can see that we are off by \$ 6,230, which is larger than the given result.

## To do
* The MAE is larger than the example given. Why?
* What does `steps_per_epoch` mean?
* Can we extract the MAE data at each epoch so that we can complete validation + parameter tuning step?
* How to disable the long warning messages?
* Is there a way to optimize? Reloading data each time wwe call data creator is kind of bad... What about putting the data in `config`?

In [15]:
# stop orca context when program finishes
stop_orca_context()

1/6 [====>.........................] - ETA: 0s - loss: 50.5688 - mae: 4.8188
(pid=3339) WARNING:tensorflow:AutoGraph could not transform <function preprocess at 0x7f5a248acb90> and will run it as-is.
(pid=3339) Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
(pid=3339) Cause: Unable to locate the source code of <function preprocess at 0x7f5a248acb90>. Note that functions defined in certain environments, like the interactive Python shell do not expose their source code. If that is the case, you should to define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.do_not_convert. Original error: could not get source code
(pid=3339) To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
6/6 [==============================] - 0s 779us/step - loss: 75.6614 - mae: 6.2302
Stopping orca co